In [7]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.9 MB/s eta 0:00:00


In [9]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [10]:
import os

# Define dataset + project folder in Drive
PROJECT_DIR = "/content/drive/MyDrive/YOLO_LicensePlate"
DATASET_DIR = f"{PROJECT_DIR}/datasets/license_plate"

# Create directories if not exist
os.makedirs(DATASET_DIR, exist_ok=True)
print("Project folder created at:", PROJECT_DIR)


Project folder created at: /content/drive/MyDrive/YOLO_LicensePlate


In [11]:
!pip install ultralytics
from ultralytics import YOLO


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [12]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("andrewmvd/car-plate-detection")

print("Path to dataset files:", path)


Path to dataset files: /kaggle/input/car-plate-detection


In [14]:



import os
import glob
import shutil
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
import cv2

# Dataset root (from kagglehub)
dataset_path = "/kaggle/input/car-plate-detection"
ann_dir = os.path.join(dataset_path, "annotations")
img_dir = os.path.join(dataset_path, "images")

# --- Robust image collection (recursive, all common extensions) ---
all_images = []
for ext in ["png", "jpg", "jpeg", "PNG", "JPG", "JPEG"]:
    all_images.extend(glob.glob(os.path.join(img_dir, f"*.{ext}")))

print(f"🔍 Found {len(all_images)} images in {img_dir}")

if len(all_images) == 0:
    raise ValueError(f"No images found in {img_dir}. Please check dataset contents.")

# Output YOLO structure
yolo_base = "/kaggle/working/car_plate_yolo"
for split in ["train", "val", "test"]:
    os.makedirs(f"{yolo_base}/images/{split}", exist_ok=True)
    os.makedirs(f"{yolo_base}/labels/{split}", exist_ok=True)

# --- Split: 70/20/10 ---
train_imgs, temp_imgs = train_test_split(all_images, test_size=0.3, random_state=42)
val_imgs, test_imgs = train_test_split(temp_imgs, test_size=1/3, random_state=42)

print(f"📊 Split -> Train: {len(train_imgs)}, Val: {len(val_imgs)}, Test: {len(test_imgs)}")

# --- Convert XML to YOLO format ---
def convert_xml_to_yolo(xml_file, img_w, img_h):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    yolo_data = []
    for obj in root.findall("object"):
        class_id = 0  # only one class: license plate
        bbox = obj.find("bndbox")
        xmin = int(bbox.find("xmin").text)
        ymin = int(bbox.find("ymin").text)
        xmax = int(bbox.find("xmax").text)
        ymax = int(bbox.find("ymax").text)

        # Convert to YOLO format
        x_center = (xmin + xmax) / 2.0 / img_w
        y_center = (ymin + ymax) / 2.0 / img_h
        width = (xmax - xmin) / img_w
        height = (ymax - ymin) / img_h

        yolo_data.append(f"{class_id} {x_center} {y_center} {width} {height}")
    return yolo_data

# --- Process images ---
def process_split(img_list, split):
    for img_path in img_list:
        file_name = os.path.basename(img_path)
        base_name = os.path.splitext(file_name)[0]
        xml_path = os.path.join(ann_dir, base_name + ".xml")

        if not os.path.exists(xml_path):
            print(f"⚠️ Skipping {file_name}: no annotation found")
            continue

        img = cv2.imread(img_path)
        if img is None:
            print(f"⚠️ Skipping {file_name}: cannot read image")
            continue
        h, w = img.shape[:2]

        yolo_labels = convert_xml_to_yolo(xml_path, w, h)

        shutil.copy(img_path, f"{yolo_base}/images/{split}/{file_name}")
        label_path = os.path.join(f"{yolo_base}/labels/{split}", base_name + ".txt")
        with open(label_path, "w") as f:
            f.write("\n".join(yolo_labels))

# Run conversion
process_split(train_imgs, "train")
process_split(val_imgs, "val")
process_split(test_imgs, "test")

# --- Summary ---
for split in ["train", "val", "test"]:
    img_count = len(os.listdir(f"{yolo_base}/images/{split}"))
    lbl_count = len(os.listdir(f"{yolo_base}/labels/{split}"))
    print(f"{split.upper()}: {img_count} images, {lbl_count} labels")

print("✅ YOLO dataset ready at:", yolo_base)



🔍 Found 433 images in /kaggle/input/car-plate-detection/images
📊 Split -> Train: 303, Val: 86, Test: 44
TRAIN: 303 images, 303 labels
VAL: 86 images, 86 labels
TEST: 44 images, 44 labels
✅ YOLO dataset ready at: /kaggle/working/car_plate_yolo


In [15]:
# Create YOLO data.yaml file
data_yaml = f"""
train: {yolo_base}/images/train
val: {yolo_base}/images/val
test: {yolo_base}/images/test

nc: 1
names: ['license_plate']
"""

yaml_path = os.path.join(yolo_base, "data.yaml")
with open(yaml_path, "w") as f:
    f.write(data_yaml)

print("✅ data.yaml created at:", yaml_path)



✅ data.yaml created at: /kaggle/working/car_plate_yolo/data.yaml


In [16]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model (nano version for fast training)
model = YOLO("yolov8n.pt")

# Train on our dataset
results = model.train(
    data="/kaggle/working/car_plate_yolo/data.yaml",
    epochs=30,
    imgsz=640,
    batch=16,
    project="/kaggle/working/yolo_runs",
    name="car_plate_train",
    verbose=False  # suppress too much log spam
)

print("✅ Training completed")


Ultralytics 8.3.191 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/car_plate_yolo/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=car_plate_train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True,

In [17]:
import pandas as pd

# Path to training logs
log_path = "/kaggle/working/yolo_runs/car_plate_train/results.csv"

# Load logs
df = pd.read_csv(log_path)

# Show accuracy (mAP50) every 5 epochs
print("📊 Accuracy (mAP50) every 5 epochs:")
for epoch in range(0, len(df), 5):
    row = df.iloc[epoch]
    print(f"Epoch {int(row['epoch'])}: mAP50 = {row['metrics/mAP50(B)']:.4f}")


📊 Accuracy (mAP50) every 5 epochs:
Epoch 1: mAP50 = 0.1618
Epoch 6: mAP50 = 0.7574
Epoch 11: mAP50 = 0.7369
Epoch 16: mAP50 = 0.8968
Epoch 21: mAP50 = 0.9228
Epoch 26: mAP50 = 0.9400
